<a href="https://colab.research.google.com/github/lbal-biomat/quality/blob/main/log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El paper en el que me guío es:

Nicholls, S. M., Quick, J. C., Tang, S., & Loman, N. J. (2019). Ultra-deep, long-read nanopore sequencing of mock microbial community standards. Gigascience, 8(5), giz043. https://academic.oup.com/gigascience/article/8/5/giz043/5486468

La idea es seguir los pasos del paper y ver que todo es reproducible, luego ensamblar con flye los últimos datos que publicaron (R10.3) y hacer el polish sobre el flye-raw (40-polishing/filtered_contigs.fasta) y sobre el assembly final (assembly.fasta) siguiendo los protocolos más recomendados, todos usando la calidad en algún punto. 

Los programas de polishing a usar son racon, medaka, marginPolish, HELEN, y homopolish. De ellos, racon y marginPolish usan información de calidad por base.

Del github de marginPolish: "If quality scores are present, the base likelihood is factored into alignment weight estimation".

Del paper de racon:  "Chunks of reads with an average quality lower than a predefined threshold are removed from the corresponding window. With this quality filter, we are able to use only high-quality parts of reads in the final consensus. Each window is then processed independently by constructing a POA graph and calling the consensus of the window. Quality values are used again during POA graph construction, where each edge is weighted by the sum of qualities of its source and destination nodes (bases; logarithmic domain)".

medaka y helen usan redes neurales recurrentes y fueron diseñados para corregir la salida de racon y marginPolish respectivamente. homopolish se basa en support-vector machines que entrena con secuencias homólogas que descarga del ncbi, y corrige principalmente indels, no missmatches. 

 <br/>

**Instalación de programas**

Instalé las versiones correspondientes al paper con conda:

```
conda create --prefix ./env_paper python=3.6 pilon=1.23 racon=1.3.2 minimap2=2.14 kraken2 checkm-genome quast=5.0.2
```
`wtdbg2` no estaba disponible en la versión 2.2, por lo que bajé los ejecutables de https://github.com/ruanjue/wtdbg2/releases/download/v2.2/wtdbg-2.2_x64_linux.tar.gz

`medaka` tampoco estaba disponible en la versión 0.5.0, por lo que lo instalé desde la fuente en un environment aparte.
Para eso descargue los archivos de https://pypi.org/project/medaka/0.5.0/#files. La versión correcta de medaka es importante porque la nueva no tiene el modelo correspondiente al basecaller que usaron en el paper.

Antes de hacer make install a medaka hay que cambiar algunas dependencias del archivo `requirements.txt`:
- numpy==1.19.2 en lugar de numpy, y pasar esa linea arriba de la de tensorflow
- sacar la version a tensorflow (la que pide, 1.x.x, ya no está disponible como libreria de python)

 <br/>

Instalé las últimas versiones de los programas en otro environment:

```
conda create --prefix ./env_nuevos flye=2.8.2 racon=1.4.13 medaka=1.2.1 minimap2=2.17
```
homopolish tiene problemas de compatibilidad con todo así que lo puse en un env aparte:

```
conda create --prefix ./hmp homopolish=0.0.1
```

marginpolish no está disponibles en conda o vía `pip`, lo cloné del github y compilé siguiendo los pasos del github.
HELEN sí está disponible vía `pip`:
```
pip install helen --user
```


 <br/>

**Datos y descarga**

Zymo Community Standards 2 (Even) Batch ZRC190633:
10 species (5 Gram-positive, 3 Gram-negative, 2 yeast): the bacteria are present at 12% and yeast at 2% (by genomic DNA)

Zymo Community Standards 2 (Log/Staggered) Batch ZRC190842
10 species (5 Gram-positive, 3 Gram-negative, 2 yeast) ranging from 10^2 - 10^8 genomic DNA abundance (total input 5 x 10^8 cells)

| Community | # of Spots    | # of Bases | Size |
| ----------|:-------------:| ----------:|-----:|
| Even      | 3,491,390     | 14.4G      | 12Gb | 
| Log       | 	3,667,480   |   16.5G    | 13.7Gb |


		  	        	      
Esos son los datos del poro 9.41 secuenciados con la plataforma GridION (para el pipeline del paper). Descargué los fastQ de https://github.com/LomanLab/mockcommunity.

En el paper también usan los datos secuenciados con PromethION. Estos son muy pesados porque tienen mucha profundidad. En principio no los voy a usar.

También están los datos de señal, en principio no los descargué, para seguir los pasos del paper estoy trabajando con los datos tal cual los de ellos.

Descargué los datos de poro R10.3 de https://nanopore.s3.climb.ac.uk/mock/Zymo-GridION-EVEN-3Peaks-R103-merged.fq.gz. Son pocos reads (1.16M). También tienen publicados datos con el poro R10, son muchos mas reads (6.74M), disponibles en https://nanopore.s3.climb.ac.uk/mock/Zymo-GridION-Even-3Peaks-Native-R10_hac_meth.fq.gz.

Descargué los assemblies de spades con reads de illumina de http://nanopore.s3.climb.ac.uk/mockcommunity/v2/Zymo-Isolates-SPAdes-Illumina.fasta.

Descagué los assemblies de referencia con pacbio de https://github.com/al-mcintyre/mCaller_analysis_scripts/tree/master/assemblies (hay disponibles 8 de los 10 organismos, faltan L. fermentum y C. neoformans). Descargué los dos que fataban refseq. No estoy segura de necesitar estos assemblies.

Descargué del SRA los reads de la secuenciación shotgun illumina de la comunidad even para el polishing con pilon: ERR2984773.

<br/>

**Algunos problemas con los programas**

Racon carga muy lento los reads comprimidos, es mejor dárselos sin comprimir. 

minimap2 y racon no se pueden usar con nohup porque la salida estándar va a parar al archivo final. Se puede poner el comando dentro de un script y llamar al script con nohup.

El primer intento de correr racon me dio el siguiente error:
```
[racon::Polisher::initialize] error: empty overlap set!
```
Hay que borrar el newline al final del archivo .paf de minimap2, y voila!

La versión 0.5.0 de medaka tiene problemas de incompatibilidad con la nueva versión de tensorflow. 
Tuve que reemplazar `tf` por `tf.compat.v1` en varios lugares de algunos scripts (todo lo que iba dando error), esos archivos fueron:
- medaka-0.5.0/venv/lib/python3.8/site-packages/keras/backend/tensorflow_backend.py
- medaka-0.5.0/venv/lib/python3.8/site-packages/medaka-0.5.0-py3.8-linux-x86_64.egg/medaka/inference.py

En el archivo medaka-0.5.0/venv/lib/python3.8/site-packages/medaka-0.5.0-py3.8-linux-x86_64.egg/medaka/common.py tuve que cambiar, en la linea 443, `raise StopIteration` por `return`. Esto es porque en las últimas versión de python (>3.7) cambió cómo se usan los generators: `StopIteration` ahora genera una excepción en lugar de simplemente terminar la vida útil del generator como antes (antes la excepción era silenciada).

Con esos cambios pude correr el programa.

La última versión de medaka (1.2.1) instalada desde conda tiene un bug en el uso de threads. Conviene dejar el default, si no termina usando todos los threads de la máquina. Lo malo es que el mapping con un solo thread es muy lento.

El manual de homopolish habla de un flag `--meta`, pero según [éste](https://github.com/ythuang0522/homopolish/issues/17#issuecomment-728980231) issue en el github de homopolish, es porque aún no hicieron el release del módulo correspondiente al flag `--meta`, por ahora lo corro sin ese flag y después veo de correrlo de nuevo.

El menor número de threads que puede usar HELEN es 8, cualquier número menor (incluso el default) hace saltar un error de que espera un "positive integer", seguramente un bug.
Aparte de eso HELEN crashea a veces y no tengo claro por qué. No es un tema de memoria.

<br/>

**Notas**

El polishing sobre el ensamblado final lo hice sobre todos los contigs, para hacer el polishing sobre el ensamblado raw primero seleccioné los 8 contigs más largo como hacen en el paper.

<br/>

**Evaluación**

Para la evaluación, uso fastmer.py y metaQUAST (plots, #missmatches).
La idea es usar las referencias de illumina para evaluar los missmatches de los 8 contigs mas largos, que corresponden al ensamblaje completo de las bacterias de la comunidad, tal como hicieron en el paper. Los missassemblies los evaluaron usando las referencias de pacbio, pero eso no es muy relevante para este trabajo.

Seleccioné los 8 contigs más largos de cada ensamblaje a evaluar:

```
infoseq --only  --length --name r10-asm.fasta | awk '{print $2, $1}' | sort -nr | head -n8 | cut -d' ' -f2 > lista_contigs_mayores
fastaUtils.pl -u r10-asm.fasta | grep --no-group-separator -A 1 -w -f lista_contigs_mayores > r10-asm-select.fa
```

Evaluación con metaQUAST:
```
metaquast.py --fragmented --min-identity 90 --min-contig 5000 --threads 28 -r ../../illumina_ref_asm/bac/ -o metaquast-selected-ill *.fasta
```